### Samuel Zeleke - CS344 - Homework 3

1. Compute the following values using the restaurant example.

    The *information gain* provided by using the price attribute as the root of the decision tree. Is it more or less 
valuable than the type or patrons attributes computed in class?

    ANSWER:
        Gain(Price) = 1 - Remainder(A)
                    = 1 - ((3/12) * B(1/3) + (2/12) * B(2/2) + (7/12) * B(3/7))
                    = 1 - (0.25 * -((1/3) * lg(1/3) + (2/3) * lg(2/3))
                           + (1/6) * -((2/2) * lg(2/2) + (0/2) * lg(0/2))
                           + (7/12) * -((3/7) * lg(3/7) + (4/7) * lg(4/7))
                      )
                    = 1 - (0.25 * -(-0.07952 -0.389975) + 
                            (1/6) * -(0) +
                            (7/12) * -(-0.52388 - 0.46135) 
                          )
                    = 1 - 0.69209
                    = 0.3079
        Since, gain from Patrons is greater than the gain for Price, asking about price is less valuable than asking 
        about Patrons. 